<a href="https://colab.research.google.com/github/ingridstefanie/byingrid/blob/main/pipeline_etl_artistas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explorando Dados de Artistas Musicais (ETL com Python)

**Contexto:** Considerando a constante evolução que acontece dentro do mercado musical, os dados desempenham um papel importante para entender o desempenho dos artistas, identificar tendências e, também, os padrões de consumo. Com isso, a análise de dados visa apoiar nas decisões estratégicas deste setor.

**Condições do Problema:**

1. Este projeto cria um *Pipeline ETL* em Python, responsável por extrair dados de artistas de um arquivo csv, transformando esses dados a partir de uma estrutura que gera descrições automáticas com IA:
  ```
| id | nome         | genero           | pais          | ano_inicio | plays_anuais  |
| -- | ------------ | ---------------- | ------------- | ---------- | ------------- |
| 1  | Taylor Swift | Pop              | EUA           | 2006       | 9.800.000.000 |
| 2  | The Weeknd   | R&B              | Canadá        | 2010       | 7.500.000.000 |
| 3  | Anitta       | Pop/Funk         | Brasil        | 2013       | 1.200.000.000 |
| 4  | BTS          | K-pop            | Coreia do Sul | 2013       | 8.900.000.000 |
| 5  | Coldplay     | Rock Alternativo | Reino Unido   | 2000       | 6.400.000.000 |


  ```
  

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [41]:
import pandas as pd

df = pd.read_csv('artistas.csv')
print("✅ Dados extraídos com sucesso!")
print(df.head())

✅ Dados extraídos com sucesso!
         id;nome;genero;pais;ano_inicio;plays_anuais
0             1;Taylor Swift;Pop;EUA;2006;9800000000
1            2;The Weeknd;R&B;Canada;2010;7500000000
2           3;Anitta;Pop/Funk;Brasil;2013;1200000000
3          4;BTS;K-pop;Coreia do Sul;2013;8900000000
4  5;Coldplay;Rock Alternativo;Reino Unido;2000;6...


## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [42]:
from openai import OpenAI
import pandas as pd
from tqdm.auto import tqdm
from dotenv import load_dotenv
import os
from google.colab import userdata


try:
    openai_api_key = userdata.get('OPENAI_API_KEY')
except userdata.SecretNotFoundError:
    openai_api_key = None


if openai_api_key:
    client = OpenAI(api_key=openai_api_key)
else:
    client = None


def extract_data(file_path):
    df = pd.read_csv(file_path, delimiter=';')
    print(f"✅ {len(df)} registros carregados de {file_path}")
    return df

def transform_data(df, client):
    if client is None:
        print("Transformação ignorada: cliente OpenAI não inicializado (chave da API ausente).")
        df["descricao"] = "Chave da API não configurada"
        return df


    def generate_ai_description(row):
        prompt = (
            f"Crie uma descrição curta (máx. 100 caracteres) "
            f"sobre o artista {row['nome']}, do gênero {row['genero']}, "
            f"que teve {row['plays_anuais']:,} reproduções no último ano."
        )

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "Você é um analista musical criativo."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=50
            )
            return response.choices[0].message.content.strip('"')

        except Exception as e:
            print(f"⚠️ Erro ao gerar descrição para {row['nome']}: {e}")
            return "Descrição não gerada"

    tqdm.pandas(desc="Gerando descrições")
    df["descricao"] = df.progress_apply(generate_ai_description, axis=1)
    return df

def load_data(df, output_path):
    """Salva o DataFrame transformado em um novo CSV."""
    df.to_csv(output_path, index=False, encoding="utf-8-sig")
    print(f"💾 Dados salvos em: {output_path}")


def main():
    input_file = "artistas.csv"
    output_file = "artistas_com_descricoes.csv"

    df = extract_data(input_file)
    df_transformed = transform_data(df, client)
    load_data(df_transformed, output_file)


if __name__ == "__main__":
    main()

✅ 5 registros carregados de artistas.csv


Gerando descrições:   0%|          | 0/5 [00:00<?, ?it/s]

⚠️ Erro ao gerar descrição para Taylor Swift: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
⚠️ Erro ao gerar descrição para The Weeknd: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
⚠️ Erro ao gerar descrição para Anitta: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insu

## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [43]:
def load_data(df, output_path):
    df.to_csv(output_path, index=False, encoding="utf-8-sig")
    print(f"💾 Dados salvos em: {output_path}")